### data-science-london-scikit-learn

https://www.kaggle.com/competitions/data-science-london-scikit-learn

### DataSet

In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from pandasql import sqldf

from pandas_profiling import ProfileReport

In [ ]:
train_df = pd.read_csv('../input/data-science-london-scikit-learn/train.csv', names=[str(i) for i in range(40)])
test_df = pd.read_csv('../input/data-science-london-scikit-learn/test.csv', names=[str(i) for i in range(40)])
train_labels_df = pd.read_csv('../input/data-science-london-scikit-learn/trainLabels.csv', names=['label'])

In [ ]:
train_df

In [ ]:
train_df_profile = ProfileReport(train_df, title='train_df', minimal=True)
test_df_profile = ProfileReport(test_df, title='test_df', minimal=True)
train_labels_df_profile = ProfileReport(train_labels_df, title='train_labels_df', minimal=True)

In [ ]:
# train_df_profile.to_notebook_iframe()

### Gaussian Model

In [ ]:
x_all = pd.concat([train_df, test_df])

In [ ]:
from sklearn.mixture import GaussianMixture, BayesianGaussianMixture

gmm = BayesianGaussianMixture(n_components=7, n_init=3)
gmm.fit(x_all)
print(list(gmm.weights_))

x_train = gmm.predict_proba(train_df)
x_test = gmm.predict_proba(test_df)

### Final Model

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(x_train, train_labels_df['label'], test_size=0.3, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
model = lgb.LGBMClassifier()
model.fit(
    X=X_train, y=y_train, 
    eval_set=[(X_train, y_train), (X_test, y_test)],
    callbacks=[
        lgb.early_stopping(stopping_rounds=10)
    ]
)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(y_test, model.predict(X_test), digits=4))

### Submit Results

In [ ]:
pred_result = model.predict(x_test)

In [ ]:
submission = []
for i, v in enumerate(pred_result):
    submission.append({
        'Id': i + 1,
        'Solution': v
    })
pd.DataFrame(submission).to_csv('submission.csv', index=False)